In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.8.5" 
JULIA_PACKAGES="IJulia Plots PyPlot JSON BenchmarkTools"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

# Install Julia
JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
rm /tmp/julia.tar.gz
echo "Julia installation complete"

# Install Packages
echo "Installing packages..."
if [ "$COLAB_GPU" = "1" ]; then
  JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
fi
for PKG in `echo $JULIA_PACKAGES`; do
echo "Installing Julia package $PKG..."
julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
done
echo "Package installation complete"

# Install kernel and rename it to "julia"
echo "Installing IJulia kernel..."
julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
  "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
mv -f $KERNEL_NAME "$KERNEL_DIR"/julia  

# Upload repository
echo "Uploading repository..."
git clone https://github.com/DavidKendall/perimeter_paper_run_simulations /content/sim
echo "Upload complete"

# Run eswa.ipynb
echo "Running notebook... get a coffee"
jupyter nbconvert \
	--to 'html' \
	--ExecutePreprocessor.allow_errors=True \
	--ExecutePreprocessor.timeout=-1 \
	--FilesWriter.build_directory=../results \
	--execute /content/code/eswa.ipynb